In [ ]:
#note_number = 21 # the 1st MIDI note corresponding to A0 (27.5Hz)
note_number = 69 # MIDI note corresponding to A4 (440Hz)
#note_number = 108 # the last MIDI note corresponding to C8 (4186Hz)

def set_piano(): 
    global piano 
    global selected_sf_name 
    global note_number 
    global originalAUDIO 
    global originalMIDI 
    global originalTXT 
    global generatedAUDIO 
    global generatedMIDI 
    global generatedTXT 
    global filename 
    global differencePNG 
    global differenceTXT 
    piano = "Mason&Hamlin" 
    selected_sf_name = "Mason&Hamlin" 
    originalAUDIO = "reference_note.wav" 
    originalMIDI = "reference_note.mid" 
    originalTXT = "reference_note.txt" 
    generatedAUDIO = "generated_note.wav" 
    generatedMIDI = "generated_note.mid" 
    generatedTXT = "generated_note.txt" 
    filename = "differece_note_" + str(note_number) 
    differencePNG = filename + ".png" 
    differenceTXT = filename + ".txt" 
    
set_piano() 
print("piano = ", piano) 
print("note_nr = ", note_number) 
print("filename = ", filename )

from IPython.display import Audio

In [ ]:
# Creating a new MIDI file

from mido import Message, MidiFile, MidiTrack, MetaMessage

# Create a new MIDI file and add a track.
mid = MidiFile(ticks_per_beat=480)
track = MidiTrack()
mid.tracks.append(track)

# Set a tempo (here, 500000 microseconds per beat corresponds to 120 BPM).
# With 480 ticks per beat, one beat lasts 0.5 sec; thus, 240 ticks equal 0.25 sec.
track.append(MetaMessage('set_tempo', tempo=500000, time=0))

# Define our note and timing parameters.
note = note_number     # MIDI note number (middle C)
note_duration = 120  # Duration (in ticks) of each note (e.g. 120 ticks = 0.25 sec / 2)
gap_duration = 120   # Gap (in ticks) between the end of one note and the start of the next.
                     # This ensures that each note-on event occurs every 240 ticks (4 times per second).

# Generate 128 note events, with velocities from 0 to 127.
first_note = True
for velocity in range(128):
    if first_note:
        # The very first note_on event starts immediately.
        track.append(Message('note_on', note=note, velocity=velocity, time=0))
        first_note = False
    else:
        # For subsequent notes, delay the note_on event by gap_duration ticks.
        track.append(Message('note_on', note=note, velocity=velocity, time=gap_duration))
    # Each note is held for note_duration ticks before turning off.
    track.append(Message('note_off', note=note, velocity=0, time=note_duration))

# Save the MIDI file.
mid.save(originalMIDI)
print(f"MIDI file '{originalMIDI}' created successfully!")


In [ ]:
#plotting MIDI notes velocities

import pretty_midi
import matplotlib.pyplot as plt
import numpy as np

# Load the MIDI file using the filename 'reference_note.mid'
midi_data = pretty_midi.PrettyMIDI(originalMIDI)

# Initialize lists to hold the times (in seconds) and velocities of note onsets
times = []
velocities = []

# Loop through all instruments in the MIDI file and extract note data
for instrument in midi_data.instruments:
    for note in instrument.notes:
        times.append(note.start)          # Record the start time of each note
        velocities.append(note.velocity)    # Record the velocity of each note

# Write the note velocities to the "reference_note.txt" file
with open(originalTXT, "w") as file:
    for v in velocities:
        file.write(f"{v}\n")

# Create two subplots arranged in one row
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Subplot 1: Scatter plot of note velocity vs. time
ax1.scatter(times, velocities, alpha=0.7, color='blue')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Velocity')
ax1.set_title('Note Velocity vs. Time')
ax1.grid(True)

# Subplot 2: Histogram of velocity values with 128 bins (0 to 127)
bins = np.arange(0, 129, 1)  # Creates bin edges from 0 to 128 (128 bins for 0-127)
ax2.hist(velocities, bins=bins, edgecolor='black', alpha=0.7, color='green')
ax2.set_xlabel('Velocity')
ax2.set_ylabel('Count')
ax2.set_title('Histogram of Velocity Values')
ax2.grid(True)

# Adjust the layout and display the plots
plt.tight_layout()
plt.show()


print(f"TXT file " + originalTXT + " generated successfully!")




In [ ]:
#rendering MIDI events with piano sounds

import subprocess
import ipywidgets as widgets
from IPython.display import display, Audio
import os


# Define available SoundFonts with your provided paths.
soundfonts = {
    "Mason&Hamlin": "/Users/user/.soundfonts/MasonHamlin-A-v7.sf2",
    "MuseScore": "/Users/user/.soundfonts/MuseScore_General.sf2",
    "4U_Steinway": "/Users/user/.soundfonts/4U-Steinway-v3.6.sf2",
    "Steinway_Chateau": "/Users/user/.soundfonts/Steinway-Chateau-Plus-Instruments-v1.7.sf2",
    "NY_S&S_Model_B": "/Users/user/.soundfonts/Dore Mark's NY S&S Model B-v5.2.sf2",
    "Nice_Steinway": "/Users/user/.soundfonts/Nice-Steinway-v3.9.sf2",
    "Fazioli_v2.5": "/Users/user/.soundfonts/Dore Mark's (SF) Fazioli-v2.5.sf2",
    "Fazioli_F308": "/Users/user/.soundfonts/Dore Mark's Fazioli F308-v3.0.sf2",
    "Yamaha_C5": "/Users/user/.soundfonts/Yamaha C5 Grand-v2.4.sf2",
    "Yamaha_S6": "/Users/user/.soundfonts/Dore Mark's Yamaha S6-v1.6.sf2",
    "Clementi_Fortepiano_1808": "/Users/user/.soundfonts/Clementi Fortepiano 1808 (Dore Mark)-v1.0.sf2"
}

# Define available MIDI files (update these paths as needed)
midi_files = {
#    "Step14": "Step_14.mid",
    "MIDI FILE": originalMIDI,
}

# Create radio buttons for SoundFont selection.
soundfont_selector = widgets.RadioButtons(
    options=list(soundfonts.keys()),
    description="SoundFont:",
    disabled=False
)

# Create radio buttons for MIDI file selection.
midi_selector = widgets.RadioButtons(
    options=list(midi_files.keys()),
    description="MIDI:",
    disabled=False
)

# Create a button widget to trigger synthesis.
synthesize_button = widgets.Button(
    description="Synthesize Audio",
    disabled=False,
    button_style='',  # Options: 'success', 'info', 'warning', 'danger'
    tooltip='Click to synthesize audio',
)

# Create an output widget to display messages.
output = widgets.Output()

def on_synthesize_button_clicked(b):
    global selected_sf_name  # Keep the variable global as requested.
    with output:
        output.clear_output()
        # Retrieve selected SoundFont and MIDI file paths.
        selected_sf_name = soundfont_selector.value
        selected_midi_name = midi_selector.value
        soundfont_path = soundfonts[selected_sf_name]
        midi_file_path = midi_files[selected_midi_name]
        
        # Verify that the files exist.
        if not os.path.exists(soundfont_path):
            print(f"SoundFont file not found: {soundfont_path}")
            return
        if not os.path.exists(midi_file_path):
            print(f"MIDI file not found: {midi_file_path}")
            return
        
        # Determine output audio file name.
        print("Selected SoundFont:", selected_sf_name)
        print("Selected MIDI:", selected_midi_name)
        print("Output audio file will be:", originalAUDIO)
        
        # Build the FluidSynth command.
        sample_rate = 44100
        command = [
            "fluidsynth",
            "-ni", 
            soundfont_path,
            midi_file_path,
            "-F", originalAUDIO,
            "-r", str(sample_rate)
        ]
        print("Running command:", " ".join(command))
        try:
            subprocess.run(command, check=True)
            print("Synthesis finished. Audio written to", originalAUDIO)
        except subprocess.CalledProcessError as e:
            print("Error during synthesis:", e)

# Bind the button click event to the callback function.
synthesize_button.on_click(on_synthesize_button_clicked)

# Display the widgets.
display(soundfont_selector, midi_selector, synthesize_button, output)

In [ ]:
# playing out 
Audio(originalAUDIO)

In [ ]:
# piano transcription from audio to MIDI using the AI model
!python example.py --audio_path={originalAUDIO} --output_midi_path={generatedMIDI}

In [ ]:
# plotting MIDI notes velocities

import mido
import matplotlib.pyplot as plt

# Specify the MIDI file name.
midi_filename = generatedMIDI  # change this if your MIDI file has a different name

# Load the MIDI file.
mid = mido.MidiFile(midi_filename)

# Determine ticks per beat from the file.
ticks_per_beat = mid.ticks_per_beat

# Try to extract the tempo from the first track.
# If no set_tempo is found, assume a default tempo (500000 microseconds per beat, i.e. 120 BPM).
tempo = 500000
for msg in mid.tracks[0]:
    if msg.type == 'set_tempo':
        tempo = msg.tempo
        break

# Iterate over all messages in time order and accumulate the absolute time (in ticks).
absolute_tick = 0
times = []       # List to hold absolute times in seconds
velocities = []  # List to hold the note velocities
notes = []       # Optional: list to hold the MIDI note numbers

for msg in mid:
    absolute_tick += msg.time  # msg.time is in ticks (delta time)
    # We only process note_on events with nonzero velocity (i.e. note start events)
    if msg.type == 'note_on' and msg.velocity > 0:
        # Convert the absolute tick time to seconds.
        time_sec = mido.tick2second(absolute_tick, ticks_per_beat, tempo)
        times.append(time_sec)
        velocities.append(msg.velocity)
        notes.append(msg.note)  # This is optional if you also want the note number

# Now create a scatter plot.
plt.figure(figsize=(10, 5))
# Here, we plot velocity (y-axis) versus time in seconds (x-axis).
plt.scatter(times, velocities, color='blue', marker='o')
plt.xlabel('Time (seconds)')
plt.ylabel('Velocity (MIDI)')
plt.title("MIDI Note Nr = " + str(note_number) + " generated velocities")

plt.grid(True)
plt.show()


In [ ]:
#rendering MIDI events with piano sounds

import subprocess
import ipywidgets as widgets
from IPython.display import display, Audio
import os


# Define available SoundFonts with your provided paths.
soundfonts = {
    "Mason&Hamlin": "/Users/user/.soundfonts/MasonHamlin-A-v7.sf2",
    "MuseScore": "/Users/user/.soundfonts/MuseScore_General.sf2",
    "4U_Steinway": "/Users/user/.soundfonts/4U-Steinway-v3.6.sf2",
    "Steinway_Chateau": "/Users/user/.soundfonts/Steinway-Chateau-Plus-Instruments-v1.7.sf2",
    "NY_S&S_Model_B": "/Users/user/.soundfonts/Dore Mark's NY S&S Model B-v5.2.sf2",
    "Nice_Steinway": "/Users/user/.soundfonts/Nice-Steinway-v3.9.sf2",
    "Fazioli_v2.5": "/Users/user/.soundfonts/Dore Mark's (SF) Fazioli-v2.5.sf2",
    "Fazioli_F308": "/Users/user/.soundfonts/Dore Mark's Fazioli F308-v3.0.sf2",
    "Yamaha_C5": "/Users/user/.soundfonts/Yamaha C5 Grand-v2.4.sf2",
    "Yamaha_S6": "/Users/user/.soundfonts/Dore Mark's Yamaha S6-v1.6.sf2",
    "Clementi_Fortepiano_1808": "/Users/user/.soundfonts/Clementi Fortepiano 1808 (Dore Mark)-v1.0.sf2"
}

# Define available MIDI files (update these paths as needed)
midi_files = {
#    "Step14": "Step_14.mid",
    "MIDI FILE": generatedMIDI,
}

# Create radio buttons for SoundFont selection.
soundfont_selector = widgets.RadioButtons(
    options=list(soundfonts.keys()),
    description="SoundFont:",
    disabled=False
)

# Create radio buttons for MIDI file selection.
midi_selector = widgets.RadioButtons(
    options=list(midi_files.keys()),
    description="MIDI:",
    disabled=False
)

# Create a button widget to trigger synthesis.
synthesize_button = widgets.Button(
    description="Synthesize Audio",
    disabled=False,
    button_style='',  # Options: 'success', 'info', 'warning', 'danger'
    tooltip='Click to synthesize audio',
)

# Create an output widget to display messages.
output = widgets.Output()

def on_synthesize_button_clicked(b):
    global selected_sf_name  # Keep the variable global as requested.
    with output:
        output.clear_output()
        # Retrieve selected SoundFont and MIDI file paths.
        selected_sf_name = soundfont_selector.value
        selected_midi_name = midi_selector.value
        soundfont_path = soundfonts[selected_sf_name]
        midi_file_path = midi_files[selected_midi_name]
        
        # Verify that the files exist.
        if not os.path.exists(soundfont_path):
            print(f"SoundFont file not found: {soundfont_path}")
            return
        if not os.path.exists(midi_file_path):
            print(f"MIDI file not found: {midi_file_path}")
            return
        
        # Determine output audio file name.
        print("Selected SoundFont:", selected_sf_name)
        print("Selected MIDI:", selected_midi_name)
        print("Output audio file will be:", generatedAUDIO)
        
        # Build the FluidSynth command.
        sample_rate = 44100
        command = [
            "fluidsynth",
            "-ni", 
            soundfont_path,
            midi_file_path,
            "-F", generatedAUDIO,
            "-r", str(sample_rate)
        ]
        print("Running command:", " ".join(command))
        try:
            subprocess.run(command, check=True)
            print("Synthesis finished. Audio written to", generatedAUDIO)
        except subprocess.CalledProcessError as e:
            print("Error during synthesis:", e)

# Bind the button click event to the callback function.
synthesize_button.on_click(on_synthesize_button_clicked)

# Display the widgets.
display(soundfont_selector, midi_selector, synthesize_button, output)

In [ ]:
# playing out
Audio(generatedAUDIO)

In [ ]:
# plotting MIDI notes velocities and their histogram

import pretty_midi
import matplotlib.pyplot as plt
import numpy as np

# Load the MIDI file using the filename 'generated_note.mid'
midi_data = pretty_midi.PrettyMIDI(generatedMIDI)

# Initialize lists to hold the times (in seconds) and velocities of note onsets
times = []
velocities = []

# Loop through all instruments in the MIDI file and extract note data
for instrument in midi_data.instruments:
    for note in instrument.notes:
        times.append(note.start)          # Record the start time of each note
        velocities.append(note.velocity)    # Record the velocity of each note

# Write the note velocities to the "reference_note.txt" file
with open(generatedTXT, "w") as file:
    for v in velocities:
        file.write(f"{v}\n")

# Create two subplots arranged in one row
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Subplot 1: Scatter plot of note velocity vs. time
ax1.scatter(times, velocities, alpha=0.7, color='blue')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Velocity')
ax1.set_title("MIDI Note Nr = " + str(note_number) + " generated velocities")
ax1.grid(True)

# Subplot 2: Histogram of velocity values with 128 bins (0 to 127)
bins = np.arange(0, 129, 1)  # Creates bin edges from 0 to 128 (128 bins for 0-127)
ax2.hist(velocities, bins=bins, edgecolor='black', alpha=0.7, color='green')
ax2.set_xlabel('Velocity')
ax2.set_ylabel('Count')
ax2.set_title('Histogram of Velocity Values')
ax2.grid(True)

# Adjust the layout and display the plots
plt.tight_layout()
plt.show()


print(f"TXT file " + generatedTXT + " generated successfully!")

In [ ]:
# comparing original velocities with those generated by the AI model and calculating Best Fit to adjust their values

import mido
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

def extract_note_data(midi_filename):
    """
    Loads a MIDI file and extracts absolute times (in seconds), velocities, 
    and MIDI note numbers for all note_on events (with nonzero velocity). 
    The times are shifted so that the first event occurs at 0 seconds.
    
    Args:
        midi_filename (str): Path to the MIDI file.
        
    Returns:
        times (list of float): Times in seconds (shifted so first event is at 0).
        velocities (list of int): Velocity values (0-127) for each note_on event.
        notes (list of int): MIDI note numbers for each note_on event.
    """
    mid = mido.MidiFile(midi_filename)
    ticks_per_beat = mid.ticks_per_beat

    # Get tempo from the first track; default to 500000 µs per beat (120 BPM) if not found.
    tempo = 500000
    for msg in mid.tracks[0]:
        if msg.type == 'set_tempo':
            tempo = msg.tempo
            break

    absolute_tick = 0
    times = []
    velocities = []
    notes = []
    
    # Iterate over messages in time order, accumulating delta times (in ticks).
    for msg in mid:
        absolute_tick += msg.time
        if msg.type == 'note_on' and msg.velocity > 0:
            time_sec = mido.tick2second(absolute_tick, ticks_per_beat, tempo)
            times.append(time_sec)
            velocities.append(msg.velocity)
            notes.append(msg.note)
    
    # Shift times so that the first event is at 0 seconds.
    if times:
        t0 = times[0]
        times = [t - t0 for t in times]
        
    return times, velocities, notes
    
# Specify your two MIDI files.
midi_filename1 =  generatedMIDI
midi_filename2 = originalMIDI

# Extract note data from both MIDI files.
times1, velocities1, notes1 = extract_note_data(midi_filename1)
times2, velocities2, notes2 = extract_note_data(midi_filename2)

# Use the same number of events from both files.
n1 = len(times1)
n2 = len(times2)
n = min(n1, n2)

# Instead of using time stamps, we create a common x-axis of note indices (0 to n-1)
common_indices = np.arange(n)
velocities1_plot = np.array(velocities1[:n])
velocities2_plot = np.array(velocities2[:n])

# Calculate the difference (generated - reference) for each event.
velocity_diff_ref = velocities1_plot - velocities2_plot

# Calculate the best fit line (y = A*x + B) for the generated_note.mid velocities.
A, B = np.polyfit(common_indices, velocities1_plot, 1)
y_fit = A * common_indices + B

# New difference computed as (best fit - reference).
velocity_diff_ref_fit = y_fit - velocities2_plot

# Calculate the fourth chart using subtraction: (generated - (best fit - reference))
fourth_values = velocities1_plot - velocity_diff_ref_fit

# Create subplots: the upper subplot for the line plots and the lower for the bar charts.
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# --- Upper subplot ---
# Plot reference velocities (black).
ax1.plot(common_indices, velocities2_plot, color='black', marker='o', linestyle='-', label=midi_filename2)
# Plot generated velocities (blue).
ax1.plot(common_indices, velocities1_plot, color='blue', marker='o', linestyle='-', label=midi_filename1)
# Plot the best-fit line (red).
ax1.plot(common_indices, y_fit, color='red', linewidth=2, label='Best Fit (generated)')
# Plot the new fourth chart (green) using subtraction with the updated label.
ax1.plot(common_indices, fourth_values, color='green', marker='o', linestyle='-', 
         label='postprocessed (generated - Best Fit + reference)')

ax1.set_ylabel('Velocity (MIDI)')
note_nr = notes1[0] if notes1 else "N/A"
#ax1.set_title("MIDI Note Nr = " + str(note_nr) + " reference, generated, best fit, and postprocessed velocities")
ax1.set_title(f"MIDI Note Nr = {note_number} reference, generated, best fit, and postprocessed velocities")

ax1.legend()
ax1.grid(True)

# Display the calculated coefficients A and B in the lower right corner.
anchored_text = AnchoredText(
    f"Best fit correction: Y = {A:.2f}X + {B:.2f}\nA = {A:.2f}\nB = {B:.2f}",
    loc='lower right',
    prop=dict(size=12, ha='left')
)
anchored_text.patch.set_boxstyle("round,pad=0.5")
anchored_text.patch.set_facecolor('white')
anchored_text.patch.set_alpha(0.5)
ax1.add_artist(anchored_text)

# --- Lower subplot ---
# We'll use a bar width that allows two bars per index.
bar_width = 0.35
x_left = common_indices - bar_width/2   # positions for blue bars: generated - reference
x_right = common_indices + bar_width/2  # positions for red bars: best fit - reference

# Bar chart 1: Difference between generated and reference velocities (blue).
ax2.bar(x_left, velocity_diff_ref, width=bar_width, color='blue', alpha=0.7, label='Generated - Reference')
# Bar chart 2: Difference between best fit and reference (red).
ax2.bar(x_right, velocity_diff_ref_fit, width=bar_width, color='red', alpha=0.7, label='Best Fit - Reference')

# Change the xlabel for the lower subplot to "Velocity (0:127)".
ax2.set_xlabel('Velocity (0:127)')
ax2.set_ylabel('Velocity Difference')
ax2.set_title("Differences in Velocities")
ax2.grid(True)
ax2.legend()

plt.tight_layout()

# Add a label "qqq" in the lower left corner of the entire figure.
plt.figtext(0.01, 0.01, "Piano:  " + piano + "", ha="left", va="bottom", fontsize=12,
            bbox=dict(facecolor='white', alpha=0.5, pad=5))


# Save the plot as a PNG image.
plt.savefig(differencePNG)
print("Plot saved as " + differencePNG)

# Save the calculated differences to a text file.
with open(differenceTXT, "w") as f:
    f.write("Index\tDiff (Generated - Reference)\tDiff (Best Fit - Reference)\n")
    for idx, d_ref, d_fit in zip(common_indices, velocity_diff_ref, velocity_diff_ref_fit):
        f.write(f"{idx}\t{d_ref}\t{d_fit}\n")
print("Calculated differences saved in " + differenceTXT)

plt.show()
